<a href="https://colab.research.google.com/github/chogerlate/Paper_implementation/blob/main/NaiveBeyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kaggle

In [ ]:
!cp /content/drive/MyDrive/colab/kaggle.json /content/kaggle.json
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           3327        103  1.0              
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                      9KB  2022-10-06 08:55:25          19785        553  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                     3MB  2022-11-16 13:52:31           2373         64  1.0              
fuarresvij/gdp-growth-around-the-globe                      

#Load Iris Dataset

In [ ]:
!kaggle datasets download -d saurabh00007/iriscsv

iriscsv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/iriscsv.zip 

Archive:  /content/iriscsv.zip
  inflating: Iris.csv                


#Import Libraries

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("/content/Iris.csv") 
df.set_index("Id",inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


In [ ]:
def create_dic(df,dict_data,text):
  for i,j in enumerate(df[text]) :
    if j in dict_data:
      pass
    else :
      dict_data.update({j:len(dict_data)})
  print(dict_data)
  return dict_data

In [ ]:
ClassId={}
ClassId=create_dic(df,ClassId,'Species')
df['Species'].replace(ClassId,inplace=True)

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


In [ ]:
df["Species"] = df['Species'].astype(float)

In [ ]:
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)
            

    def predict(self, X):
        y_pred = [self._Bestposterior(x) for x in X]
        return np.array(y_pred)

    def _Bestposterior(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = self._priors[idx]
            posterior = np.prod(self._GNB(idx, x))
            posterior = posterior * prior
            posteriors.append(posterior)

        # return class with the highest posterior
        return self._classes[np.argmax(posteriors)]

    def _GNB(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [ ]:
X = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,[-1]].to_numpy().flatten()
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=2022
)

nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

Naive Bayes classification accuracy 0.9666666666666667
